## **오차역전파법**

편미분 대신 Backpropagation을 사용할때의 장점?

가중치 매개변수의 기울기를 구할때 정직하게 수치 미분을 하는것이 아니고,

노드별 미리 정해진 미분을 상수와 같이 사용하므로 계산에 있어서 매우 효율적이다.

## **곱셈 계층 구현하기**

In [1]:
import numpy as np

In [2]:
class MulLayer: # 계산그래프의 곱셈 노드
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y): # 순전파
        self.x = x
        self.y = y
        out = x * y

        return out

    def backward(self, dout): # 역전파
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy

In [3]:
# 사과 쇼핑 예시에서 순전파 구현

apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer() 
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [4]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


## **덧셈 계층 구현하기**

In [5]:
class AddLayer: # 계산 그래프의 덧셈 노드
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1 # 덧셈 노드는 상류의 값을 여과 없이 하류로 내보낸다.
        dy = dout * 1
        return dx, dy

In [6]:
# 사과/귤 쇼핑 예시에서 계산 그래프 구현

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## **ReLU 계층 구현하기**

In [7]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0) # 활성화 함수 Relu의 수식
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [8]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [9]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


## **Sigmoid 계층 구현하기**

In [10]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out # Sigmoid의 정해진 미분 dout * y(1-y)

        return dx

## **Affine 계층 구현하기**

신경망의 순전파때 수행하는 행렬의 곱을 기하학에서 **어파인 변환(affine transform)**이라고 한다.  예) $ Y = X \cdot\ W + B$

In [11]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self. b # affine transform
        
        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T) # 행렬의 역전파
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

## **Softmax-with-Loss 계층 구현하기**

신경망의 출력(Softmax의 출력)이 정답 레이블과 가까워지도록 가중치 매개변수의 값을 조정하는 것이 신경망 학습의 목적이다.

따라서 신경망의 출력과 정답 레이블의 오차를 효율적으로 앞 계층에 전달 해야한다. -> 역전파

In [12]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실
        self.y = None # softmax의 출력
        self.t = None # 정답 레이블(원-핫 벡터)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size # 오차를 앞 계층에 전달한다. (y - t)

        return dx

## **신경망 학습의 전체 그림**

**전제**
    
　신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 한다. 신경망 학습은 다음과 같이 4단계로 수행한다.

**1단계 - 미니배치**

　훈련 데이터 중 일부를 무작위로 가져온다.. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실함수 값을 줄이는 것이 목표이다.

**2단계 - 기울기 산출**

　미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시한다.

**3단계 - 매개변수 갱신**

　가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.

**4단계 - 반복**

　1~3단계를 반복한다.

## **오차역전파법을 적용한 신경망 구현하기**

In [13]:
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/drive/MyDrive/밑바닥부터시작하는딥러닝/deep-learning-from-scratch-master'

import sys
sys.path.append(my_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict() # 신경망의 계층을 '순서가 있는' 딕셔너리에 보관한다. 즉 딕셔너리에 추가한 순서를 기억한다.
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

## **오차역전파법으로 구한 기울기 검증하기**

오차역전파법이 완벽해 보이지만, 사실은 구현하기가 복잡하기 때문에 종종 실수가 생기기도 한다.

따라서 연산이 느리지만 구현이 쉬운 수치미분을 통한 결과값과 오차역전파법을 통한 결과값을 비교하여 오차역전파법을 정확히 구현했는지 확인한다.

이를 **기울기 확인(gradient check)**이라고 한다.

In [18]:
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:3.5092118536502753e-10
b1:1.9991539724481057e-09
W2:5.447185221690474e-09
b2:1.3973421384844143e-07


## **오차역전파법을 사용한 학습 구현하기**

In [17]:
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.11591666666666667 0.1159
0.9044333333333333 0.9072
0.9229166666666667 0.9262
0.9358666666666666 0.936
0.9432833333333334 0.941
0.9515166666666667 0.9481
0.9558166666666666 0.9525
0.9585 0.957
0.9626 0.9606
0.9669666666666666 0.9648
0.9672166666666666 0.9626
0.9719 0.9679
0.9734 0.9673
0.9755833333333334 0.97
0.97595 0.9691
0.97815 0.972
0.97955 0.9713


# **정리**

* **계산 그래프를 이용하면 계산 과정을 시각적으로 파악할 수 있다.**
* **계산 그래프의 노드는 국소적 계산으로 구성된다. 국소적 계산을 조합해 전체 계산을 구성한다.**
* **계산 그래프의 순전파는 통상의 계산을 수행한다. 한편, 계산그래프의 역전파로는 각 노드의 미분을 구할 수 있다.**
* **신경망의 구성 요소를 계층으로 구현하여 기울기를 효율적으로 계산할 수 있다(오차역전파법).**
* **수치 미분과 오차역전파법의 결과를 비교하면 오차역전파법의 구현에 잘못이 없는지 확인할 수 있다(기울기 확인),**